In [1]:
from funcoes import get_filtered_df, get_amino

In [2]:
df = get_filtered_df('Human (Jurkat lymphoma T cells).xlsx', 300, ['PROTEIN', 'LENGTH', 'UNIPROT_ID', 'Tm_(C)', 'ADDITIVES'])
df

,index,PROTEIN,LENGTH,UNIPROT_ID,Tm_(C),ADDITIVES
0,2,GTPase NRas,189.0,P01111,62.18,NaN
1,6,Rho-related GTP-binding protein RhoB,196.0,P62745,51.06,NaN
2,7,Prefoldin subunit 1,122.0,O60925,56.02,NaN
3,11,Ribonuclease H1,286.0,O60930,43.11,NaN
4,17,Short coiled-coil protein,159.0,Q9UIL1,53.87,NaN
...,...,...,...,...,...,...
5951,28186,Gamma-glutamylaminecyclotransferase,153.0,Q9BVM4,51.63,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5952,28220,Prostamide/prostaglandin F synthase,198.0,Q8TBF2,54.11,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5953,28276,NEDD8-conjugating enzyme UBE2F,185.0,Q969M7,53.66,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)
5954,28352,Rab-like protein 2A,228.0,Q9UBK7,48.03,0.67% NP40 (octylphenoxypoly(ethyleneoxy)ethanol)


In [3]:
#for id_ in df['UNIPROT_ID']:
#    print(id_, get_amino(id_))